## 1️⃣ Setup & Dependencies (2-3 min)

**Note:** The `datasets` package will automatically download the MS MARCO hard negatives dataset (~2-3 GB) during first training run.

In [ ]:
# Install required packages
pip install -q transformers datasets accelerate scipy torch==2.8.0 --upgrade

In [ ]:
# Verify GPU (should be A100-SXM4-40GB for Colab Pro)
!nvidia-smi

## 2️⃣ Mount Google Drive & Upload Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**⚠️ IMPORTANT**: Upload these files to your Google Drive first:
- `anomalous_embedding_ultimate.py`
- `training_monitor.py` (optional, for enhanced monitoring)

Put them in: `/content/drive/MyDrive/anomalous_embedding/`

In [ ]:
# Copy files from Drive to Colab runtime
!mkdir -p /content/anomalous
!cp /content/drive/MyDrive/anomalous_embedding/anomalous_embedding_ultimate.py /content/anomalous/
!cp /content/drive/MyDrive/anomalous_embedding/training_monitor.py /content/anomalous/ 2>/dev/null || echo "training_monitor.py not found (optional)"

# Change to working directory
%cd /content/anomalous

# Verify files
!ls -lh

## 3️⃣ Check Configuration (Optional)

In [ ]:
# Show parameter estimation for M456 (default)
!python anomalous_embedding_ultimate.py --mode param_estimate --size M456

In [ ]:
# Optional: Check M700 if you want to try it
!python anomalous_embedding_ultimate.py --mode param_estimate --size M700

## 4️⃣ Start Training - M456 (6-8 hours)

**Default configuration:**
- Size: M456 (458M params)
- Epochs: 3
- All components enabled (spectral, anchor64, bridge, matryoshka, angular)
- Automatic monitoring every 500 steps
- Early stopping enabled
- Checkpoints saved in `checkpoints_custom/`

**📥 Dataset Download:**
The training will automatically download the MS MARCO hard negatives dataset (~2-3 GB) from Hugging Face at first run. This takes 5-10 minutes and happens only once (cached afterwards).

In [ ]:
# RECOMMENDED: Full M456 training (6-8h)
!python anomalous_embedding_ultimate.py --mode train --epochs 3

### Alternative: M700 for SOTA (12 hours)

**Only if you have time and want maximum performance:**

In [ ]:
# Alternative: M700 training (12h, target STS >0.840)
# !python anomalous_embedding_ultimate.py --size M700 --mode train --epochs 3

### Quick Test Run (1-2 hours)

**For debugging or quick experiments:**

In [ ]:
# Quick 1-epoch test (1-2h)
# !python anomalous_embedding_ultimate.py --mode train --epochs 1

## 5️⃣ Backup Checkpoints to Drive

**Run this after training completes or periodically during training:**

In [ ]:
# Create backup directory in Drive
!mkdir -p /content/drive/MyDrive/anomalous_checkpoints

# Copy all checkpoints to Drive
!cp -r checkpoints_custom/* /content/drive/MyDrive/anomalous_checkpoints/

# List backed up files
!ls -lh /content/drive/MyDrive/anomalous_checkpoints/

print("✅ Checkpoints backed up to Google Drive!")

## 6️⃣ Evaluation

**After training completes, evaluate the model:**

In [ ]:
# Quick evaluation with best checkpoint
!python anomalous_embedding_ultimate.py --mode eval --checkpoint checkpoints_custom/best_sts.pt

# Expected output:
# Recall@10: 0.XXX | STS (Spearman): 0.825+

## 7️⃣ Comprehensive Evaluation Suite (Optional)

**For detailed benchmarking against SOTA models:**

In [ ]:
# Upload anomalous_eval_suite.py if you have it
!cp /content/drive/MyDrive/anomalous_embedding/anomalous_eval_suite.py /content/anomalous/ 2>/dev/null || echo "eval suite not found"

# Quick evaluation (subset of data)
!python anomalous_eval_suite.py --checkpoint checkpoints_custom/best_sts.pt --size M456 --quick

# Full evaluation (takes longer)
# !python anomalous_eval_suite.py --checkpoint checkpoints_custom/best_sts.pt --size M456

## 8️⃣ Extract Embeddings (Optional)

**Test the model with custom text:**

In [ ]:
# Extract embeddings for custom texts
!python anomalous_embedding_ultimate.py \
    --mode extract \
    --checkpoint checkpoints_custom/best_sts.pt \
    --texts "Quantum mechanics reveals hidden patterns" "Machine learning transforms data"

## 9️⃣ Ablation Studies (Optional)

**Test individual component contributions:**

In [ ]:
# Test without spectral attention (1 epoch quick test)
# !python anomalous_embedding_ultimate.py --no-spectral --mode train --epochs 1

# Test without anchor64
# !python anomalous_embedding_ultimate.py --no-anchor64 --mode train --epochs 1

# Test minimal configuration (no auxiliary components)
# !python anomalous_embedding_ultimate.py --no-spectral --no-anchor64 --no-bridge --no-matry --mode train --epochs 1

## 📊 Monitor Training Progress

**Look for these indicators during training:**

### Good Signs ✅
```
[MONITOR] STS-B: 0.XXX | Target: 0.825 | Progress: XX.X%
[MONITOR] ✓ New best STS: 0.XXX
```

### Warning Signs ⚠️
```
⚠️ Gradient explosion detected
⚠️ Embedding collapse detected
⚠️ No improvement (5/5)
⚠️ EARLY STOPPING TRIGGERED
```

### Expected Timeline:
- **Step 500**: First evaluation, STS ~0.60-0.70
- **Step 1000**: STS ~0.75-0.80
- **Step 2000**: STS ~0.80-0.82
- **End of training**: STS >0.825 (target achieved!)

## 🔧 Troubleshooting

### Out of Memory (OOM)

If you get OOM errors with M456:

1. **Reduce batch size** (edit the config in the .py file):
   - Change `batch_train = 32` → `batch_train = 24`
   - Or try `batch_train = 16`

2. **Use gradient checkpointing** (if available)

3. **Switch to M456** if you're trying M700

### Training Too Slow

Check GPU type:
```python
!nvidia-smi
```

Should show: **A100-SXM4-40GB**

If not A100, training will be much slower.

### Files Not Found

Make sure you uploaded:
- `anomalous_embedding_ultimate.py` to `/content/drive/MyDrive/anomalous_embedding/`
- (Optional) `training_monitor.py` to the same location

### Import Errors

Re-run the installation cell:
```python
!pip install -q transformers datasets accelerate scipy torch --upgrade
```

## 🎯 Success Criteria

### M456 (Default)
- ✅ STS-B > 0.825 (competitive with BGE-base)
- ✅ Training completes <8h on A100
- ✅ Fits in Colab Pro (40GB)
- ✅ No crashes or OOM

### M700 (SOTA Target)
- ✅ STS-B > 0.840 (competitive with GTE-large)
- ✅ Training completes <12h on A100

## 📝 Notes

- **Checkpoints** are saved every 2000 steps in `checkpoints_custom/`
- **Best model** is saved as `best_sts.pt` when STS improves
- **Early stopping** triggers after 5 evaluations without improvement
- **Backup regularly** to Google Drive to avoid data loss
- **Colab disconnects** after ~12 hours idle - keep the tab open!

---

## 🚀 Quick Start Summary

**For most users, just run these cells in order:**

1. Install dependencies
2. Mount Drive
3. Copy files
4. Start training (M456, 6-8h)
5. Backup checkpoints
6. Evaluate

**Expected result**: STS-B > 0.825 ✅

---

**File**: `anomalous_embedding_ultimate.py`

**Status**: ✅ Production Ready

**Best for**: Colab Pro with A100 40GB

🎯 **Ready to train!**